<a href="https://colab.research.google.com/github/yarusx/cat_vs_dogo/blob/main/dogs_breeds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import pathlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q /content/drive/MyDrive/Dog.zip

In [ ]:
data_dir = pathlib.Path('/content/Dog')

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (160, 160)

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  labels='inferred',
  label_mode='int',
  class_names = ['pug', 'basset_hound', 'shiba_inu', 'leonberger', 'saint_bernard', 'miniature_pinscher', 'wheaten_terrier', 'scottish_terrier', 'pomeranian', 'german_shorthaired', 'english_setter', 'newfoundland', 'boxer', 'great_pyrenees', 'samoyed', 'japanese_chin', 'english_cocker_spaniel', 'havanese', 'american_pit_bull_terrier', 'keeshond', 'american_bulldog', 'chihuahua', 'beagle', 'yorkshire_terrier', 'staffordshire_bull_terrier'],
  validation_split=0.2,
  subset="training",
  seed=51,
  image_size=IMG_SIZE,
  batch_size=BATCH_SIZE)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  labels='inferred',
  label_mode='int',
  class_names = ['pug', 'basset_hound', 'shiba_inu', 'leonberger', 'saint_bernard', 'miniature_pinscher', 'wheaten_terrier', 'scottish_terrier', 'pomeranian', 'german_shorthaired', 'english_setter', 'newfoundland', 'boxer', 'great_pyrenees', 'samoyed', 'japanese_chin', 'english_cocker_spaniel', 'havanese', 'american_pit_bull_terrier', 'keeshond', 'american_bulldog', 'chihuahua', 'beagle', 'yorkshire_terrier', 'staffordshire_bull_terrier'],
  validation_split=0.2,
  subset="validation",
  seed=51,
  image_size=IMG_SIZE,
  batch_size=BATCH_SIZE)

Found 4990 files belonging to 25 classes.
Using 3992 files for training.
Found 4990 files belonging to 25 classes.
Using 998 files for validation.


In [ ]:
class_names = train_dataset.class_names

# plt.figure(figsize=(10, 10))
# for images, labels in train_dataset.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")

In [ ]:
# val_batches = tf.data.experimental.cardinality(validation_dataset)
# test_dataset = validation_dataset.take(val_batches // 5)
# validation_dataset = validation_dataset.skip(val_batches // 5)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
# test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
])

In [ ]:
# for image, _ in train_dataset.take(1):
#   plt.figure(figsize=(10, 10))
#   first_image = image[0]
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
#     plt.imshow(augmented_image[0] / 255)
#     plt.axis('off')

In [ ]:
#preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
preprocess_input = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset=-1)

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 5, 5, 1280)


In [ ]:
base_model.trainable = False

In [ ]:
global_avg_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_avg = global_avg_layer(feature_batch)
print(feature_batch_avg.shape)

(32, 1280)


In [ ]:
prediction_layer = tf.keras.layers.Dense(37)
prediction_batch = prediction_layer(feature_batch_avg)
print(prediction_batch.shape)

(32, 37)


In [ ]:
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_avg_layer(x)
x = tf.keras.layers.Dropout(0.33)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
loss0, accuracy0 = model.evaluate(validation_dataset)
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

32/32 [==============================] - 2s 73ms/step - loss: 3.9040 - accuracy: 0.0200
initial loss: 3.90
initial accuracy: 0.02


In [ ]:
initial_epochs = 1
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)
val_acc = history.history['val_accuracy']

125/125 [==============================] - 15s 121ms/step - loss: 2.0118 - accuracy: 0.4409 - val_loss: 1.3108 - val_accuracy: 0.7355


In [ ]:
while np.mean(val_acc)*100 < 90.0:
  initial_epochs = 5
  history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)
  
  val_acc = history.history['val_accuracy']
  
model.save("dogs_breeds.pb")

from google.colab import files
files.download("dogs_breeds.pb")

Epoch 1/5
125/125 [==============================] - 14s 113ms/step - loss: 0.3992 - accuracy: 0.8750 - val_loss: 0.2983 - val_accuracy: 0.8938
Epoch 2/5
125/125 [==============================] - 14s 113ms/step - loss: 0.4009 - accuracy: 0.8745 - val_loss: 0.2982 - val_accuracy: 0.8988
Epoch 3/5
125/125 [==============================] - 14s 113ms/step - loss: 0.4006 - accuracy: 0.8660 - val_loss: 0.2985 - val_accuracy: 0.8938
Epoch 4/5
125/125 [==============================] - 14s 112ms/step - loss: 0.3753 - accuracy: 0.8818 - val_loss: 0.2994 - val_accuracy: 0.8948
Epoch 5/5
125/125 [==============================] - 14s 112ms/step - loss: 0.3745 - accuracy: 0.8823 - val_loss: 0.2962 - val_accuracy: 0.8958
Epoch 1/5
125/125 [==============================] - 14s 112ms/step - loss: 0.3802 - accuracy: 0.8785 - val_loss: 0.2934 - val_accuracy: 0.8988
Epoch 2/5
125/125 [==============================] - 14s 113ms/step - loss: 0.3917 - accuracy: 0.8783 - val_loss: 0.2924 - val_accuracy:

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/dogs_breeds.pb')

In [ ]:
from keras.preprocessing.image import load_img, img_to_array

# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(160, 160))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 160, 160, 3)
	# center pixel data
	#img = preprocess_input(img)
	return img

img = load_image('/content/drive/MyDrive/dayana_1.JPG')

In [ ]:
labels = ['pug', 'basset_hound', 'shiba_inu', 'leonberger', 'saint_bernard', 'miniature_pinscher', 'wheaten_terrier', 'scottish_terrier', 'pomeranian', 'german_shorthaired', 'english_setter', 'newfoundland', 'boxer', 'great_pyrenees', 'samoyed', 'japanese_chin', 'english_cocker_spaniel', 'havanese', 'american_pit_bull_terrier', 'keeshond', 'american_bulldog', 'chihuahua', 'beagle', 'yorkshire_terrier', 'staffordshire_bull_terrier']

In [ ]:
prediction = model.predict(img, batch_size=None,)
prediction = tf.nn.softmax(prediction)
prediction = np.argmax(prediction, axis=1)
print(prediction)

In [ ]:
plt.figure()
plt.imshow(img[0]/255)
plt.title(labels[int(prediction)])

In [ ]:
# #Retrieve a batch of images from the test set
# image_batch, label_batch = test_dataset.as_numpy_iterator().next()
# predictions = model.predict_on_batch(image_batch)

# # Apply a sigmoid since our model returns logits
# predictions = tf.nn.softmax(predictions)
# #print(predictions.shape)
# predictions = np.argmax(predictions, axis=1)

# print('Predictions:\n', predictions)
# print('Labels:\n', label_batch)

# plt.figure(figsize=(10, 10))
# for i in range(9):
#   ax = plt.subplot(3, 3, i + 1)
#   plt.imshow(image_batch[i].astype("uint8"))
#   plt.title(class_names[predictions[i]])
#   plt.axis("off")